In [11]:
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
origin_data = pd.read_csv('train.csv')
data = origin_data.values.tolist()

In [5]:
#统计每个节点连接了多少条边
edge_nums = {}
for i in range(len(data)):
    if data[i][0] not in edge_nums.keys():
        edge_nums[data[i][0]] = 0
    if data[i][1] not in edge_nums.keys():
        edge_nums[data[i][1]] = 0
    edge_nums[data[i][0]] += 1
    edge_nums[data[i][1]] += 1

In [8]:
#将每个节点连接的边数从大到小排列
edge_num = sorted(edge_nums.items(), key = lambda x:x[1], reverse = True)

In [12]:
#构造无向图
G = nx.Graph()
for i in range(len(data)): 
    G.add_edge(data[i][0], data[i][1])
G.to_undirected()

In [13]:
#计算PageRank值
pagerank_list = nx.pagerank(G, alpha=1)
pageRank = sorted(pagerank_list.items(),key=lambda x:x[1],reverse=True)

In [16]:
#提取PageRank值最高的前20个数据
pagerank_top20 = []
for i in range(20):
    pagerank_top20.append(pageRank[i])

In [39]:
#提取PageRank值最高的前20个数据的id
id_list = []
for i in range(20):
    id_list.append(pagerank_top20[i][0])

In [40]:
#根据PageRank值最高的前20个数据的id寻找它们相应的连接边数
edge_top20 = []
for i in range(len(edge_num)):
    if edge_num[i][0] in id_list:
        edge_top20.append(edge_num[i])

In [45]:
#计算每个节点连接的边数占前20个节点连接总边数的比例
total = 0
for i in range(20):
    total += edge_top20[i][1]
prb = {}
for i in range(20):
    prb[edge_top20[i][0]] = edge_top20[i][1] / total

In [52]:
#用了比较复杂的方式汇总数据
#最终数据key:id values:列表
#列表第一个元素为PageRank值，第二个元素为节点连接边数，第三个元素为比列
final_data = {}
for i in range(20):
    if pagerank_top20[i][0] not in final_data.keys():
        final_data[pagerank_top20[i][0]] = []
        final_data[pagerank_top20[i][0]].append(pagerank_top20[i][1])
        for j in range(20):
            if edge_top20[j][0] == pagerank_top20[i][0]:
                final_data[pagerank_top20[i][0]].append(edge_top20[j][1])
                for key in prb.keys():
                    if key == pagerank_top20[i][0]:
                        final_data[pagerank_top20[i][0]].append(prb[key])

In [54]:
#写入json格式
import json
bubble_diagram = json.dumps(final_data, ensure_ascii = False)
fp1 = open('bubble_diagram.json', 'w+')
fp1.write(bubble_diagram)
fp1.close()